In [2]:
import re
regexp = re.compile(r'^(\d+)(_\w+)?.(sql|py)')

In [4]:
regexp.match('0001.p')

In [6]:
!ls ..

dev	    mi-pkg	  setup.py		Vagrantfile  vdi.egg-info
gtasks	    Pipfile	  tests			vagrant.sh
migrations  Pipfile.lock  typed_contextmanager	vdi


In [7]:
from pathlib import Path

In [8]:
Path('../migrations')

PosixPath('../migrations')

In [16]:

list(_8.glob('*.[sql|py]'))

[]

In [17]:
import sys
sys.path

['/home/pwtail/projects/vdi2/dev',
 '/home/pwtail/.local/share/virtualenvs/vdi2-W7tVCc89/lib/python37.zip',
 '/home/pwtail/.local/share/virtualenvs/vdi2-W7tVCc89/lib/python3.7',
 '/home/pwtail/.local/share/virtualenvs/vdi2-W7tVCc89/lib/python3.7/lib-dynload',
 '/usr/lib64/python3.7',
 '/usr/lib/python3.7',
 '',
 '/home/pwtail/.local/share/virtualenvs/vdi2-W7tVCc89/lib/python3.7/site-packages',
 '/home/pwtail/projects/vdi2',
 '/home/pwtail/projects/vdi2/gtasks',
 '/home/pwtail/projects/vdi2/typed_contextmanager',
 '/home/pwtail/projects/vdi2/mi-pkg',
 '/home/pwtail/.local/share/virtualenvs/vdi2-W7tVCc89/lib/python3.7/site-packages/IPython/extensions',
 '/home/pwtail/.ipython']

In [18]:
import hey

ModuleNotFoundError: No module named 'hey'

In [23]:
sys.path.insert(0, '/home/pwtail/projects/vdi2/migrations')

In [24]:
sys.path

['/home/pwtail/projects/vdi2/migrations',
 'migrations',
 '/home/pwtail/projects/vdi2/dev',
 '/home/pwtail/.local/share/virtualenvs/vdi2-W7tVCc89/lib/python37.zip',
 '/home/pwtail/.local/share/virtualenvs/vdi2-W7tVCc89/lib/python3.7',
 '/home/pwtail/.local/share/virtualenvs/vdi2-W7tVCc89/lib/python3.7/lib-dynload',
 '/usr/lib64/python3.7',
 '/usr/lib/python3.7',
 '',
 '/home/pwtail/.local/share/virtualenvs/vdi2-W7tVCc89/lib/python3.7/site-packages',
 '/home/pwtail/projects/vdi2',
 '/home/pwtail/projects/vdi2/gtasks',
 '/home/pwtail/projects/vdi2/typed_contextmanager',
 '/home/pwtail/projects/vdi2/mi-pkg',
 '/home/pwtail/.local/share/virtualenvs/vdi2-W7tVCc89/lib/python3.7/site-packages/IPython/extensions',
 '/home/pwtail/.ipython']

In [25]:
import hey

In [1]:
from vdi.tasks import admin

In [2]:
from g_tasks import g

In [3]:
g.use_threadlocal(1)

In [4]:
await admin.AddNode()

In [1]:
url = 'https://bootstrap.pypa.io/get-pip.py'

In [2]:
from vdi.tasks import admin
from g_tasks import g
g.use_threadlocal(1)

In [3]:
await admin.DownloadDiskImage(
    url=url, out='py'
)

In [4]:
!ls

cached_prop.ipynb  http_client.ipynb  tasks.ipynb	 ws.ipynb
contextvar.ipynb   import_disk.ipynb  transaction.ipynb
graphql.ipynb	   py		      Untitled.ipynb


In [5]:
!code py